In [4]:
import pandas as pd
df = pd.read_csv("C:\\Users\\tanis\\Downloads\\archive (4)\\dating_app_behavior_dataset.csv")
df.head()

,gender,sexual_orientation,location_type,income_bracket,education_level,interest_tags,app_usage_time_min,app_usage_time_label,swipe_right_ratio,swipe_right_label,likes_received,mutual_matches,profile_pics_count,bio_length,message_sent_count,emoji_usage_rate,last_active_hour,swipe_time_of_day,match_outcome
0,Prefer Not to Say,Gay,Urban,High,Bachelor’s,"Fitness, Politics, Traveling",52,Moderate,0.60,Optimistic,173,23,4,44,75,0.36,13,Early Morning,Mutual Match
1,Male,Bisexual,Suburban,Upper-Middle,No Formal Education,"Languages, Fashion, Parenting",279,Extreme User,0.56,Optimistic,107,7,3,301,35,0.42,0,Morning,Chat Ignored
2,Non-binary,Pansexual,Suburban,Low,Master’s,"Movies, Reading, DIY",49,Moderate,0.41,Optimistic,91,27,2,309,33,0.41,1,After Midnight,Date Happened
3,Genderfluid,Gay,Metro,Very Low,Postdoc,"Coding, Podcasts, History",185,Extreme User,0.32,Balanced,147,6,5,35,5,0.07,21,Morning,No Action
4,Male,Bisexual,Urban,Middle,Bachelor’s,"Clubbing, Podcasts, Cars",83,High,0.32,Balanced,94,11,1,343,34,0.11,22,After Midnight,One-sided Like


Now , as we can see there are many word type data's in these columns so we will use label encoders for them and also normalize the data , remove outliers and remove null values

In [5]:
df.dropna(inplace=True)

In [6]:
def remove_outliers_iqr(data):
    numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns
    for col in numeric_cols:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        data = data[(data[col] >= lower) & (data[col] <= upper)]
    return data

df = remove_outliers_iqr(df)

In [8]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
scaler = MinMaxScaler()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.drop('match_outcome', errors='ignore')
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [9]:
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    if col != 'match_outcome':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

In [10]:
if df['match_outcome'].dtype == 'object':
    target_encoder = LabelEncoder()
    df['match_outcome'] = target_encoder.fit_transform(df['match_outcome'])

In [11]:
print(df.head())

   gender  sexual_orientation  location_type  income_bracket  education_level  \
0       4                   3              5               0                1   
1       2                   1              4               4                6   
2       3                   5              4               1                5   
3       1                   3              0               6                8   
4       2                   1              5               3                1   

   interest_tags  app_usage_time_min  app_usage_time_label  swipe_right_ratio  \
0          11212            0.173333                     5               0.60   
1          17396            0.930000                     2               0.56   
2          22666            0.163333                     5               0.41   
3           6250            0.616667                     2               0.32   
4           5430            0.276667                     3               0.32   

   swipe_right_label  like

Now we will test it with different algorithms to check which one is better 

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [12]:
target_encoder = None
if df['match_outcome'].dtype == 'object':
    target_encoder = LabelEncoder()
    df['match_outcome'] = target_encoder.fit_transform(df['match_outcome'])

In [13]:
X = df.drop('match_outcome', axis=1)
y = df['match_outcome']

In [16]:
#Random Forest
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.10213322600120749

Classification Report:
               precision    recall  f1-score   support

           0       0.10      0.13      0.11       973
           1       0.11      0.14      0.12       964
           2       0.10      0.11      0.11       984
           3       0.10      0.10      0.10       969
           4       0.10      0.09      0.10      1017
           5       0.10      0.10      0.10      1023
           6       0.10      0.10      0.10       965
           7       0.09      0.07      0.08      1033
           8       0.12      0.10      0.11      1058
           9       0.10      0.07      0.08       952

    accuracy                           0.10      9938
   macro avg       0.10      0.10      0.10      9938
weighted avg       0.10      0.10      0.10      9938



In [17]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression(max_iter=1000, random_state=42)  
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.09981887703763333

Classification Report:
               precision    recall  f1-score   support

           0       0.09      0.19      0.13       973
           1       0.10      0.21      0.14       964
           2       0.09      0.04      0.06       984
           3       0.09      0.13      0.11       969
           4       0.11      0.05      0.07      1017
           5       0.11      0.17      0.14      1023
           6       0.10      0.04      0.06       965
           7       0.15      0.01      0.01      1033
           8       0.10      0.15      0.12      1058
           9       0.03      0.00      0.00       952

    accuracy                           0.10      9938
   macro avg       0.10      0.10      0.08      9938
weighted avg       0.10      0.10      0.08      9938



C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
#SVM
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)  
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.10303884081304085

Classification Report:
               precision    recall  f1-score   support

           0       0.10      0.51      0.17       973
           1       0.12      0.06      0.08       964
           2       0.11      0.14      0.13       984
           3       0.10      0.16      0.12       969
           4       0.00      0.00      0.00      1017
           5       0.00      0.00      0.00      1023
           6       0.11      0.18      0.14       965
           7       0.00      0.00      0.00      1033
           8       0.00      0.00      0.00      1058
           9       0.00      0.00      0.00       952

    accuracy                           0.10      9938
   macro avg       0.05      0.11      0.06      9938
weighted avg       0.05      0.10      0.06      9938



C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

In [19]:
#Neural Networks
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
num_classes = len(set(y))
if num_classes == 2:
    output_activation = 'sigmoid'
    loss_fn = 'binary_crossentropy'
else:
    output_activation = 'softmax'
    loss_fn = 'sparse_categorical_crossentropy'
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1 if num_classes == 2 else num_classes, activation=output_activation)
])
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.1, verbose=1)
y_pred_prob = model.predict(X_test)
if num_classes == 2:
    y_pred = (y_pred_prob > 0.5).astype("int32").flatten()
else:
    y_pred = y_pred_prob.argmax(axis=1)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.0976 - loss: 920.7261 - val_accuracy: 0.1011 - val_loss: 3.6766
Epoch 2/30
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1014 - loss: 4.9258 - val_accuracy: 0.0933 - val_loss: 2.3037
Epoch 3/30
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.1031 - loss: 2.3030 - val_accuracy: 0.0936 - val_loss: 2.3031
Epoch 4/30
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1040 - loss: 2.3030 - val_accuracy: 0.1034 - val_loss: 2.3031
Epoch 5/30
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0988 - loss: 2.3041 - val_accuracy: 0.0941 - val_loss: 2.3029
Epoch 6/30
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1017 - loss: 2.3023 - val_accuracy: 0.0938 - val_loss: 2.3029
Epoch 7/30
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1012 - loss: 2.3031 - val_accuracy: 0.1031 - val_loss: 2.3034
Epoch 8/30
1118/1118 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0999 - loss: 2.3024 

C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

In [20]:
#XGBoost
from xgboost import XGBClassifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')  
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [15:23:44] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.10293821694505936

Classification Report:
               precision    recall  f1-score   support

           0       0.10      0.12      0.11       973
           1       0.10      0.11      0.10       964
           2       0.12      0.13      0.13       984
           3       0.09      0.10      0.10       969
           4       0.11      0.10      0.10      1017
           5       0.10      0.10      0.10      1023
           6       0.10      0.10      0.10       965
           7       0.09      0.08      0.09      1033
           8       0.10      0.10      0.10      1058
           9       0.11      0.10      0.11       952

    accuracy                           0.10      9938
   macro avg       0.10      0.10      0.10      9938
weighted avg       0.10      0.10      0.10      9938



In [21]:
#Hyperparameters XGBoost
model = XGBClassifier(
    n_estimators=300,        
    learning_rate=0.05,     
    max_depth=5,           
    subsample=0.8,             
    colsample_bytree=0.8,      
    gamma=1,                   
    reg_lambda=1.5,            
    reg_alpha=0.5,            
    use_label_encoder=False,
    eval_metric='mlogloss',
    random_state=42
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [15:24:51] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.09851076675387402

Classification Report:
               precision    recall  f1-score   support

           0       0.09      0.12      0.11       973
           1       0.10      0.11      0.10       964
           2       0.11      0.14      0.13       984
           3       0.10      0.11      0.10       969
           4       0.09      0.08      0.09      1017
           5       0.11      0.12      0.11      1023
           6       0.09      0.09      0.09       965
           7       0.09      0.06      0.07      1033
           8       0.09      0.09      0.09      1058
           9       0.09      0.07      0.08       952

    accuracy                           0.10      9938
   macro avg       0.10      0.10      0.10      9938
weighted avg       0.10      0.10      0.10      9938



In [22]:
#Hyperparameter Neural Network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
num_classes = len(set(y))
if num_classes == 2:
    final_activation = 'sigmoid'
    loss_function = 'binary_crossentropy'
    output_units = 1
else:
    final_activation = 'softmax'
    loss_function = 'sparse_categorical_crossentropy'
    output_units = num_classes
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(output_units, activation=final_activation)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss_function,
    metrics=['accuracy']
)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_split=0.15,
    callbacks=[early_stop],
    verbose=1
)
y_pred_prob = model.predict(X_test)
if num_classes == 2:
    y_pred = (y_pred_prob > 0.5).astype("int32").flatten()
else:
    y_pred = y_pred_prob.argmax(axis=1)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Epoch 1/100


C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1056/1056 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.0997 - loss: 2.5920 - val_accuracy: 0.0964 - val_loss: 2.3067
Epoch 2/100
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0984 - loss: 2.3373 - val_accuracy: 0.0963 - val_loss: 2.3069
Epoch 3/100
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0992 - loss: 2.3165 - val_accuracy: 0.1005 - val_loss: 2.3074
Epoch 4/100
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1012 - loss: 2.3107 - val_accuracy: 0.0966 - val_loss: 2.3053
Epoch 5/100
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1018 - loss: 2.3063 - val_accuracy: 0.1026 - val_loss: 2.3036
Epoch 6/100
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.1014 - loss: 2.3052 - val_accuracy: 0.0951 - val_loss: 2.3054
Epoch 7/100
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1018 - loss: 2.3045 - val_accuracy: 0.1020 - val_loss: 2.3033
Epoch 8/100
1056/1056 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1022 - loss: 2.3047 - val_

C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\tanis\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classificati

As conclusion , we can see not always hyperparameters will increase accuracy and no matter which algorithm you use , if the dataset is not good enough then it wont give good accuracy no matter how hard you try , just like in our case we reached max 10% accuracy which clearly points to underfitting which we tried to fix by using hyperparameters but we couldnt and it mainly failed due to the use of a lot of features and label encoding them all , it confused the machine making it crash and not capable of performing to their peak best.